## Remaining Trials
This notebook is useful to estimate the remaining time required for a set of optimizations to reach a given number of trials.

In [ ]:
%reload_ext autoreload
%autoreload 2

from IPython.core import ultratb

ultratb.VerboseTB.tb_highlight = "bg:#3e0054"

In [ ]:
import pandas as pd

from syftr.configuration import cfg

STUDY_NAMES = [
    "silver1--in-sample--bright_hf--earth_science",
    "silver1--in-sample--bright_hf--economics",
    "silver1--in-sample--bright_hf--pony",
    "silver1--in-sample--bright_hf--psychology",
    "silver1--in-sample--bright_hf--robotics",
    "silver1--in-sample--bright_hf--sustainable_living",
]

N_TRIALS_FOR_ESTIMATE = 20
N_TARGET = 2400

STORAGE = cfg.database.get_optuna_storage()

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
def get_extrapolation(df_trials, n_target, n_trials_for_estimate):
    df = df_trials[-n_trials_for_estimate:]
    duration = (df.iloc[-1]["datetime_complete"] - df.iloc[0]["datetime_complete"]) / n_trials_for_estimate
    x = [df.iloc[-1]["datetime_complete"]]
    y = [df.index[-1]]
    for _ in range(1, n_target-y[0]+1):
        x.append(x[-1] + duration)
        y.append(y[-1] + 1)
    return x, y

In [ ]:
import matplotlib.pyplot as plt
import optuna
import pandas as pd

plt.figure(figsize=(10, 6))

for study_name in STUDY_NAMES:
    study = optuna.load_study(study_name=study_name, storage=STORAGE)
    df_trials: pd.DataFrame = study.trials_dataframe()
    try:
        df_trials = df_trials[df_trials["state"] == "COMPLETE"]
        df_trials = df_trials.sort_values("datetime_complete")
        df_trials.reset_index(drop=True, inplace=True)
        x = df_trials["datetime_complete"]
        y = df_trials.index
        plt.plot(x, y, label=study_name)
        x, y = get_extrapolation(df_trials, n_target=N_TARGET, n_trials_for_estimate=N_TRIALS_FOR_ESTIMATE)
        finish_date = f"{x[-1]:%A, %B %d at %H:%M}"
        print(f"{study_name} is expected to reach {N_TARGET} trials on {finish_date} (in {x[-1] - x[0]})")
        plt.plot(x, y, color="gray", linestyle="--", linewidth=0.5, alpha=0.5)
        plt.axhline(y=N_TARGET, color="gray", linestyle="--", linewidth=0.5, alpha=0.5)
    except:
        pass

plt.title(f"Optimization Trials with Extrapolation based on last {N_TRIALS_FOR_ESTIMATE} Trials")
plt.ylabel("Number of Optimization Trials")
plt.xticks(rotation=45)
plt.legend()